<!--NOTEBOOK_HEADER-->
*This notebook contains material from [PyRosetta](https://RosettaCommons.github.io/PyRosetta);
content is available [on Github](https://github.com/RosettaCommons/PyRosetta.notebooks.git).*

<!--NAVIGATION-->
< [Side Chain Conformations and Dunbrack Energies](http://nbviewer.jupyter.org/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/06.01-Side-Chain-Conformations-and-Dunbrack-Energies.ipynb) | [Contents](toc.ipynb) | [Index](index.ipynb) | [Design](http://nbviewer.jupyter.org/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/06.03-Design.ipynb) ><p><a href="https://colab.research.google.com/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/06.02-Side-chain-packing.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a>

# Side-Chain Packing
Keywords: PackerTask, standard_packer_task(), restrict_to_repacking(), temporarily_fix_everything(), temporarily_set_pack_residue(), PackRotamersMover(), SwitchResidueTypeSetMover(), MoveMap

## Monte Carlo

In [ ]:
# Notebook setup
import sys
if 'google.colab' in sys.modules:
    !pip install pyrosettacolabsetup
    import pyrosettacolabsetup
    pyrosettacolabsetup.setup()
    print ("Notebook is set for PyRosetta use in Colab.  Have fun!")

**Make sure you are in the directory with the pdb files:**

`cd google_drive/My\ Drive/student-notebooks/`

In [4]:
# From previous section:
from pyrosetta import *
from pyrosetta.teaching import *
pyrosetta.init()
pose = pose_from_pdb("inputs/1YY8.clean.pdb")
start_pose = Pose()
start_pose.assign(pose)
scorefxn = get_fa_scorefxn()

core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.Release.python36.mac r208 2019.04+release.fd666910a5e fd666910a5edac957383b32b3b4c9d10020f34c1 http://www.pyrosetta.org 2019-01-22T15:55:37
core.init: command: PyRosetta -ex1 -ex2aro -database /Users/kathyle/Computational Protein Prediction and Design/PyRosetta4.Release.python36.mac.release-208/pyrosetta/database
core.init: 'RNG device' seed mode, using '/dev/urandom', seed=-356895598 seed_offset=0 real_seed=-356895598
core.init.random: RandomGenerator:init: Normal mode, seed=-356895598 RG_type=mt19937
core.import_pose.import_pose: File '1YY8.clean.pdb' automatically determined to be of type PDB
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CG  on residue ARG 18
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CD  on residue ARG 18
core.conformation.Conformation: [ WARNING ] missing heavyatom:  NE  on residue ARG 18
core.conformation.Conformation: [ W

Side-chain packing can be done in a Monte Carlo search routine that iteratively swaps rotamers of a random residue and tests each move using the Metropolis criterion. Rosetta has such a routine pre-packaged as a `Mover` that carries out a simulated annealing search each time it is applied. The specific scope of the packing is specified in a `PackerTask` object, which is similar to a `MoveMap` in that it specifies degrees of freedom. We can specify via commands or from an input file our settings for a `PackerTask`.

Create a `PackerTask` as follows. This will set the task to allow packing only of residue 49:

```
task_pack = standard_packer_task(pose)
task_pack.restrict_to_repacking()
task_pack.temporarily_fix_everything()
task_pack.temporarily_set_pack_residue(49, True)
```

In [5]:
### BEGIN SOLUTION
task_pack = standard_packer_task(pose)
task_pack.restrict_to_repacking() # only allows rotamers of current residue
task_pack.temporarily_fix_everything()
task_pack.temporarily_set_pack_residue(49, True)
### END SOLUTION

core.pack.task: Packer task: initialize from command line()


The default task allows any amino acid residue to be swapped in for another; that is, it would simulate a protein variant as a result of mutation. This would be useful for protein design but not for side-chain packing. `restrict_to_repacking()` only allows rotamers from the current residue at that position to be used.

We can confirm our settings using (note how only one amino acid is allowed at each position):
```
print(task_pack)
```

In [6]:
### BEGIN SOLUTION
print(task_pack)
### END SOLUTION

#Packer_Task

resid	pack?	design?	allowed_aas
1	FALSE	FALSE	ASP:NtermProteinFull
2	FALSE	FALSE	ILE
3	FALSE	FALSE	LEU
4	FALSE	FALSE	LEU
5	FALSE	FALSE	THR
6	FALSE	FALSE	GLN
7	FALSE	FALSE	SER
8	FALSE	FALSE	PRO
9	FALSE	FALSE	VAL
10	FALSE	FALSE	ILE
11	FALSE	FALSE	LEU
12	FALSE	FALSE	SER
13	FALSE	FALSE	VAL
14	FALSE	FALSE	SER
15	FALSE	FALSE	PRO
16	FALSE	FALSE	GLY
17	FALSE	FALSE	GLU
18	FALSE	FALSE	ARG
19	FALSE	FALSE	VAL
20	FALSE	FALSE	SER
21	FALSE	FALSE	PHE
22	FALSE	FALSE	SER
23	FALSE	FALSE	
24	FALSE	FALSE	ARG
25	FALSE	FALSE	ALA
26	FALSE	FALSE	SER
27	FALSE	FALSE	GLN
28	FALSE	FALSE	SER
29	FALSE	FALSE	ILE
30	FALSE	FALSE	GLY
31	FALSE	FALSE	THR
32	FALSE	FALSE	ASN
33	FALSE	FALSE	ILE
34	FALSE	FALSE	HIS,HIS_D
35	FALSE	FALSE	TRP
36	FALSE	FALSE	TYR
37	FALSE	FALSE	GLN
38	FALSE	FALSE	GLN
39	FALSE	FALSE	ARG
40	FALSE	FALSE	THR
41	FALSE	FALSE	ASN
42	FALSE	FALSE	GLY
43	FALSE	FALSE	SER
44	FALSE	FALSE	PRO
45	FALSE	FALSE	ARG
46	FALSE	FALSE	LEU
47	FALSE	FALSE	LEU
48	FALSE	FALSE	ILE
49	TRUE	FALSE	LYS
50	FALSE	FALS

We now can construct a `PackRotamersMover`:

```
pack_mover = PackRotamersMover(scorefxn, task_pack)
```

In [7]:
### BEGIN SOLUTION
pack_mover = PackRotamersMover(scorefxn, task_pack)
### END SOLUTION

Apply the `PackMover` above to your pose with the `.apply()` method.

```
pack_mover.apply(pose)
```

__Question:__ Now what are the χ angles of K49? Which rotamer is this? What is the Dunbrack energy?

In [8]:
### BEGIN SOLUTION
pack_mover.apply(pose)

for i in range(1, 5):
    print(pose.chi(i, 49))
    
scorefxn(pose)
energies = pose.energies()
print(energies.residue_total_energies(49))
### END SOLUTION

core.pack.pack_rotamers: built 9 rotamers at 1 positions.
core.pack.interaction_graph.interaction_graph_factory: Instantiating DensePDInteractionGraph


__Question:__ What is the new total energy of K49? Why did Rosetta pick this rotamer? Answer this in terms of the components of the score function and in terms of the residues with which K49 interacts.

## Packing for Refinement


Side-chain packing can be used when converting a pose from centroid to full-atom mode, and it is used extensively in full-atom refinement calculations. Let’s examine how packing improves scores.

Create a centroid-representation model for RecA protein domain 2 (PDB ID: 2REB) using the `SwitchResidueTypeSetMover`. Save that centroid “decoy” so that we can compare several basic refinement steps.

```
cen_ras = pose_from_file("6Q21_A.pdb")
switch = SwitchResidueTypeSetMover("centroid")
switch.apply(cen_ras)
```

In [11]:
### BEGIN SOLUTION
cen_ras = pose_from_file("inputs/6Q21_A.pdb")
switch = SwitchResidueTypeSetMover("centroid")
switch.apply(cen_ras)
### END SOLUTION

core.import_pose.import_pose: File '6Q21_A.pdb' automatically determined to be of type PDB
core.chemical.GlobalResidueTypeSet: Finished initializing centroid residue type set.  Created 62 residue types
core.chemical.GlobalResidueTypeSet: Total time to initialize 0.038653 seconds.


Load another `ras` and keep it in full-atom representation. Save this starting configuration for future use. Score the pose with the standard centroid score function.

__Question:__ Why is the score so high?

```
ras = pose_from_file("6Q21_A.pdb")
start_ras = Pose()
start_ras.assign(ras)
scorefxn(ras)
```

In [12]:
### BEGIN SOLUTION
ras = pose_from_file("inputs/6Q21_A.pdb")
start_ras = Pose()
start_ras.assign(ras)
scorefxn(ras)
### END SOLUTION

core.import_pose.import_pose: File '6Q21_A.pdb' automatically determined to be of type PDB


1215.729069796814

Create a default `PackRotamersMover` with a `PackerTask` that allows all residues to vary χ angles. Create a test pose from your start pose and pack the side chains.

__Question:__ What is the new pose score?

```
test_ras = Pose()
test_ras.assign(start_ras)

task_pack = standard_packer_task(test_ras)
task_pack.restrict_to_repacking()
task_pack.temporarily_fix_everything()
task_pack.temporarily_set_pack_residue(49, True)

pack_mover = PackRotamersMover(scorefxn, task_pack)
pack_mover.apply(test_ras)
```

In [13]:
### BEGIN SOLUTION
test_ras = Pose()
test_ras.assign(start_ras)

task_pack = standard_packer_task(test_ras)
task_pack.restrict_to_repacking()
task_pack.temporarily_fix_everything()
task_pack.temporarily_set_pack_residue(49, True)

pack_mover = PackRotamersMover(scorefxn, task_pack)
pack_mover.apply(test_ras)
### END SOLUTION

core.pack.task: Packer task: initialize from command line()
core.pack.pack_rotamers: built 15 rotamers at 1 positions.
core.pack.interaction_graph.interaction_graph_factory: Instantiating DensePDInteractionGraph


Reset the test pose to the start configuration. Create a `MoveMap` that allows χ angles but not φ/ψ/ω angles to vary. Confirm the `MoveMap` by printing it. Create a `MinMover` using the Davidson-Fletcher-Powell minimization scheme by applying the method `min_type("dfpmin")` to your mover. Apply the `MinMover` and rescore the pose.

__Question:__ How does this energy compare?

```
test_ras.assign(start_ras)
mm = MoveMap()
mm.set_chi(True)
mm.set_bb(False)
print(mm)

min_mover = MinMover()
min_mover.set_movemap(mm)
min_mover.score_function(scorefxn)
min_mover.min_type("dfpmin")
print(min_mover)

print(scorefxn(test_ras))
min_mover.apply(test_ras)
print(scorefxn(test_ras))
```

In [14]:
### BEGIN SOLUTION
test_ras.assign(start_ras)
mm = MoveMap()
mm.set_chi(True)
mm.set_bb(False)
print(mm)

min_mover = MinMover()
min_mover.set_movemap(mm)
min_mover.score_function(scorefxn)
min_mover.min_type("dfpmin")
print(min_mover)

print(scorefxn(test_ras))
min_mover.apply(test_ras)
print(scorefxn(test_ras))
### END SOLUTION


-------------------------------
  resnum     Type  TRUE/FALSE 
-------------------------------
 DEFAULT      BB     FALSE
 DEFAULT      SC      TRUE
 DEFAULT      NU     FALSE
 DEFAULT  BRANCH     FALSE
-------------------------------
 jumpnum     Type  TRUE/FALSE 
-------------------------------
 DEFAULT     JUMP    FALSE
-------------------------------
  resnum  atomnum     Type  TRUE/FALSE 
-------------------------------
 DEFAULT               PHI    FALSE
 DEFAULT             THETA    FALSE
 DEFAULT                 D    FALSE
 DEFAULT               RB1    FALSE
 DEFAULT               RB2    FALSE
 DEFAULT               RB3    FALSE
 DEFAULT               RB4    FALSE
 DEFAULT               RB5    FALSE
 DEFAULT               RB6    FALSE
-------------------------------


Mover name: MinMover, Mover type: MinMover, Mover current tag:NoTag
Minimization type:	dfpmin
Scorefunction:		ref2015
Score tolerance:	0.01
Nb list:		True
Deriv check:		False
Movemap:

---------------------------

Again, reset the test pose to the starting configuration. Apply the packer and then minimize on the χ angles.

__Question:__ Now what is the final score?

```
test_ras.assign(start_ras)
print(scorefxn(ras))
pack_mover.apply(ras)
print(scorefxn(ras))
min_mover.apply(ras)
print(scorefxn(ras))
```

In [ ]:
### BEGIN SOLUTION
test_ras.assign(start_ras)
print(scorefxn(ras))
pack_mover.apply(ras)
print(scorefxn(ras))
min_mover.apply(ras)
print(scorefxn(ras))
### END SOLUTION

For fun, you might examine the individual residue energies to find the residues most responsible for the score changes. Typically, a small number of residues may make clashes that can be resolved using the χ angle minimization, which allows off-rotamer side-chain conformations.

<!--NAVIGATION-->
< [Side Chain Conformations and Dunbrack Energies](http://nbviewer.jupyter.org/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/06.01-Side-Chain-Conformations-and-Dunbrack-Energies.ipynb) | [Contents](toc.ipynb) | [Index](index.ipynb) | [Design](http://nbviewer.jupyter.org/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/06.03-Design.ipynb) ><p><a href="https://colab.research.google.com/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/06.02-Side-chain-packing.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a>